In [1]:
library(MASS)
library(truncnorm)


bases <- function(s, n, type) {
  # {cos(ns), sin(ns)} with 1-D s
  if (type == 1) {
    m <- round((n - 1) / 2)
    Phi <- c(1/sqrt(2) * rep(1, length(s)), cos(1:m * s), sin(1:m * s)) / sqrt(pi)
    Phi <- matrix(Phi, ncol = 1)
  }
  
  return(Phi)
}

deriv_bases <- function(s, n, type) {
  # The derivative of the bases: phi_i(s), i is from 1 to m
  # s: variable
  # n: number of bases
  # {cos(ns), sin(ns)} with 1-D s
  if (type == 1) {
    m <- round((n - 1) / 2)
    Phi_pri <- c(0, -(1:m), 1:m) * c(0, sin(1:m * s), cos(1:m * s)) / sqrt(pi)
    Phi_pri <- matrix(Phi_pri, ncol = 1)
  }
  
  return(Phi_pri)
}

deriv2_bases <- function(s, n, type) {
  # The second derivative of bases: phi_i(s), i is from 1 to n
  # s: variable
  # n: number of bases
  # {cos(ns), sin(ns)} with 1-D s
  if (type == 1) {
    m <- round((n - 1) / 2)
    coeff <- -c(0, -(1:m), 1:m)^2
    basis_functions <- c(0, cos(1:m * s), sin(1:m * s))
    Phi_pri2 <- (coeff * basis_functions) / sqrt(pi)
    Phi_pri2 <- matrix(Phi_pri2, ncol = 1)
  }
  
  return(Phi_pri2)
}


r <- function(s, beta, lam, nl, sig, k) {
beta * cos(k * s)^3 - lam * ((1 - nl) * s + nl * sin(s)^2) * (-3 * k * cos(k * s)^2 * sin(k * s)) - 
  sig^2 / 2 * (3 * 2 * cos(k * s) * (k * sin(k * s))^2 - 3 * k^2 * cos(k * s)^3)
}


In [2]:
method <- function(num_traj,num_batch,batch,N,sig=1,beta=10,T=3){
     # Dynamics parameters
      lam <- 0.05;  k <- 1
      nl <- 0

      # Time discretization
      dt <- 0.1 
      # Discount factor
      gama <- exp(-beta * dt)
    
######################################## training data     
      s_record <- matrix(0, nrow = num_traj, ncol = T)
     # Generate trajectory
    
    s_record[, 1] <- rtruncnorm(num_traj,  a = -pi, b = pi, mean = 0, sd = 0.1)
    for (ti in 2:T) {
      Et <- s_record[, ti - 1] * exp(lam * dt)
      Vt <- (exp(2 * lam * dt) - 1) * sig^2 / (2 * lam)
      s_record[, ti] <- Et + rnorm(num_traj, sd = sqrt(Vt))
    }
######################################## testing data
      x_test <- seq(-pi, pi, length.out = N) 
      # Calculate the exact value function
      y_test <- cos(seq(-pi, pi, length.out = N) * k)^3
########################################
      # Algorithm
    m <- 4
    n <- 1 + 2 * m  # Number of bases

    A_DE1 <- matrix(0, n, n)
    A_DE2 <- matrix(0, n, n)
    A_BE <- matrix(0, n, n)
    b_traj <- numeric(n)
    b_traj2 <- numeric(n)
########################################    
    s_batch_1 <- s_record
    
    for (k2 in 1:num_traj) {
        s <- s_batch_1[k2, ]
        for (i in 1:(T - 1)) {
          st <- s[i]
          stp1 <- s[i + 1]
          Phi <- bases(st, n, 1)
          Phi_pri <- deriv_bases(st, n, 1)
          Phi_2pri <- deriv2_bases(st, n, 1)

          A_BE <- A_BE + Phi %*% (t(Phi)- gama %*% t(bases(stp1, n, 1)) ) /dt 
          A_DE1 <- A_DE1 + Phi %*% (beta %*% t(Phi) - ((stp1 - st) / dt) %*% t(Phi_pri) -
                                    (1 / 2 / dt) %*% (stp1 - st)^2 %*% t(Phi_2pri))
          b_traj <- b_traj + Phi %*% r(st, beta, lam, nl, sig, k)

          if (i <= T - 2) {
            stp2 <- s[i + 2]


            A_DE2 <- A_DE2 + Phi %*% (beta  %*%  t(Phi) - (((-1 / 2) %*% (stp2 - st) + 2 %*% (stp1 - st)) / dt) %*% t(Phi_pri) -  (1 / 2 / dt) %*% ((-1 / 2) %*% (stp2 - st)^2 + 2 %*% (stp1 - st)^2) %*% t(Phi_2pri))
            b_traj2 <- b_traj2 + Phi %*% r(st, beta, lam, nl, sig, k)
          }
        }
    }
      A_DE1 <- A_DE1 / (T - 1) / num_traj * 2 * pi
      A_DE2 <- A_DE2 / (T - 2) / num_traj * 2 * pi
      A_BE <- A_BE / (T - 1) / num_traj * 2 * pi
      b_traj <- b_traj / (T - 1) / num_traj * 2 * pi
      b_traj2 <- b_traj2 / (T - 2) / num_traj * 2 * pi
      
      # Calculate the parameter 

      th_DE1 <- ginv(A_DE1) %*% b_traj 
      th_DE2 <- ginv(A_DE2) %*% b_traj2
      th_BE <- ginv(A_BE) %*% b_traj
      
      v_DE1 <- numeric(N)
      v_DE2 <- numeric(N)
      v_BE <- numeric(N)
      i <- 1
      for (s in x_test) {
        v_DE1[i] <- sum(th_DE1 * bases(s, n, 1))
        v_DE2[i] <- sum(th_DE2 * bases(s, n, 1))
        v_BE[i] <- sum(th_BE * bases(s, n, 1))
        i <- i + 1
      }
    
    prediction_1_1 <- v_DE1
    prediction_1_2  <- v_DE2
    prediction_1_rl <-  v_BE
    
    er_1_1 <-(y_test - prediction_1_1)^2
    er_1_2 <- (y_test -prediction_1_2)^2
    er_1_rl <- (y_test - prediction_1_rl)^2
########################################
    
    v_avg_de1 <- matrix(0, nrow = N, ncol = 1)
    v_avg_de2 <- matrix(0, nrow = N, ncol = 1)
    v_avg_rl <- matrix(0, nrow = N, ncol = 1)
    for (kkk in 1:num_batch) {
      idx <- sample(1:num_traj, batch, replace = FALSE)
      s_batch <- s_record[idx, ]
        
     for (k2 in 1:batch) {
        s <- s_batch[k2, ]
        for (i in 1:(T - 1)) {

          st <- s[i]
          stp1 <- s[i + 1]
          Phi <- bases(st, n, 1)
          Phi_pri <- deriv_bases(st, n, 1)
          Phi_2pri <- deriv2_bases(st, n, 1)

          A_BE <- A_BE + Phi %*% (t(Phi)- gama %*% t(bases(stp1, n, 1)) ) /dt 
          A_DE1 <- A_DE1 + Phi %*% (beta %*% t(Phi) - ((stp1 - st) / dt) %*% t(Phi_pri) -
                                    (1 / 2 / dt) %*% (stp1 - st)^2 %*% t(Phi_2pri))
          b_traj <- b_traj + Phi %*% r(st, beta, lam, nl, sig, k)

          if (i <= T - 2) {
            stp2 <- s[i + 2]


            A_DE2 <- A_DE2 + Phi %*% (beta  %*%  t(Phi) - (((-1 / 2) %*% (stp2 - st) + 2 %*% (stp1 - st)) / dt) %*% t(Phi_pri) -  (1 / 2 / dt) %*% ((-1 / 2) %*% (stp2 - st)^2 + 2 %*% (stp1 - st)^2) %*% t(Phi_2pri))
            b_traj2 <- b_traj2 + Phi %*% r(st, beta, lam, nl, sig, k)
          }
        }
      }
        
      A_DE1 <- A_DE1 / (T - 1) / num_traj * 2 * pi
      A_DE2 <- A_DE2 / (T - 2) / num_traj * 2 * pi
      A_BE <- A_BE / (T - 1) / num_traj * 2 * pi
      b_traj <- b_traj / (T - 1) / num_traj * 2 * pi
      b_traj2 <- b_traj2 / (T - 2) / num_traj * 2 * pi
      
      # Calculate the parameter 
      th_DE1 <- ginv(A_DE1) %*% b_traj 
      th_DE2 <- ginv(A_DE2) %*% b_traj2
      th_BE <- ginv(A_BE) %*% b_traj#solve(A_BE, b_traj)
      
      # Plot the approximated value function
      v_DE1 <- numeric(N)
      v_DE2 <- numeric(N)
      v_BE <- numeric(N)
      i <- 1
      for (s in  x_test) {
        v_DE1[i] <- sum(th_DE1 * bases(s, n, 1))
        v_DE2[i] <- sum(th_DE2 * bases(s, n, 1))
        v_BE[i] <- sum(th_BE * bases(s, n, 1))
        i <- i + 1
      }
      v_avg_de1[, 1] <- v_avg_de1[, 1] + v_DE1
      v_avg_de2[, 1] <- v_avg_de2[, 1] + v_DE2
      v_avg_rl[, 1] <- v_avg_rl[, 1] + v_BE  
    }
    
    prediction_2_1 <- v_avg_de1[, 1] / num_batch
    prediction_2_2  <- v_avg_de2[, 1] / num_batch
    prediction_2_rl <-  v_avg_rl[, 1] / num_batch
    
    er_2_1 <-(y_test - prediction_2_1)^2
    er_2_2 <- (y_test -prediction_2_2)^2
    er_2_rl <- (y_test - prediction_2_rl)^2
      
#############################################  
    v_avg_de1 <- matrix(0, nrow = N, ncol = 1)
    v_avg_de2 <- matrix(0, nrow = N, ncol = 1)
    v_avg_rl <- matrix(0, nrow = N, ncol = 1)
    for (kkk in 1:num_batch) {
      idx <- sample(1:num_traj, batch, replace = TRUE)
      s_batch <- s_record[idx, ]
        
     for (k2 in 1:batch) {
        s <- s_batch[k2, ]
        for (i in 1:(T - 1)) {

          st <- s[i]
          stp1 <- s[i + 1]
          Phi <- bases(st, n, 1)
          Phi_pri <- deriv_bases(st, n, 1)
          Phi_2pri <- deriv2_bases(st, n, 1)


          A_BE <- A_BE + Phi %*% (t(Phi)- gama %*% t(bases(stp1, n, 1)) ) /dt 
          A_DE1 <- A_DE1 + Phi %*% (beta %*% t(Phi) - ((stp1 - st) / dt) %*% t(Phi_pri) -
                                    (1 / 2 / dt) %*% (stp1 - st)^2 %*% t(Phi_2pri))
          b_traj <- b_traj + Phi %*% r(st, beta, lam, nl, sig, k)

          if (i <= T - 2) {
            stp2 <- s[i + 2]


            A_DE2 <- A_DE2 + Phi %*% (beta  %*%  t(Phi) - (((-1 / 2) %*% (stp2 - st) + 2 %*% (stp1 - st)) / dt) %*% t(Phi_pri) -  (1 / 2 / dt) %*% ((-1 / 2) %*% (stp2 - st)^2 + 2 %*% (stp1 - st)^2) %*% t(Phi_2pri))
            b_traj2 <- b_traj2 + Phi %*% r(st, beta, lam, nl, sig, k)
          }
        }
      }
      A_DE1 <- A_DE1 / (T - 1) / num_traj * 2 * pi
      A_DE2 <- A_DE2 / (T - 2) / num_traj * 2 * pi
      A_BE <- A_BE / (T - 1) / num_traj * 2 * pi
      b_traj <- b_traj / (T - 1) / num_traj * 2 * pi
      b_traj2 <- b_traj2 / (T - 2) / num_traj * 2 * pi
      
      # Calculate the parameter 
      th_DE1 <- ginv(A_DE1) %*% b_traj 
      th_DE2 <- ginv(A_DE2) %*% b_traj2
      th_BE <- ginv(A_BE) %*% b_traj
      
      # Plot the approximated value function
      v_DE1 <- numeric(N)
      v_DE2 <- numeric(N)
      v_BE <- numeric(N)
      i <- 1
      for (s in  x_test) {
        v_DE1[i] <- sum(th_DE1 * bases(s, n, 1))
        v_DE2[i] <- sum(th_DE2 * bases(s, n, 1))
        v_BE[i] <- sum(th_BE * bases(s, n, 1))
        i <- i + 1
      }
      v_avg_de1[, 1] <- v_avg_de1[, 1] + v_DE1
      v_avg_de2[, 1] <- v_avg_de2[, 1] + v_DE2
      v_avg_rl[, 1] <- v_avg_rl[, 1] + v_BE  
    }
    
    prediction_3_1 <- v_avg_de1[, 1] / num_batch
    prediction_3_2  <- v_avg_de2[, 1] / num_batch
    prediction_3_rl <-  v_avg_rl[, 1] / num_batch
    
    er_3_1 <-(y_test - prediction_3_1)^2
    er_3_2 <- (y_test -prediction_3_2)^2
    er_3_rl <- (y_test - prediction_3_rl)^2
    
    return(list(
  prediction_1 = matrix(c(prediction_1_1, prediction_2_1, prediction_3_1),ncol=3),
  prediction_2 = matrix(c(prediction_1_2, prediction_2_2, prediction_3_2),ncol=3),
  prediction_rl = matrix(c(prediction_1_rl, prediction_2_rl, prediction_3_rl),ncol=3),
  predict_error_1 = matrix(c(er_1_1,er_2_1, er_3_1),ncol=3),
  predict_error_2 = matrix(c(er_1_2,er_2_2, er_3_2),ncol=3),
  predict_error_rl = matrix(c(er_1_rl,er_2_rl, er_3_rl),ncol=3)
        
))
    
    }



In [3]:
experiment_1 <- function(N,num_traj, num_batch, batch,sig,times,beta=10){
  MSE_matrix_1 <- matrix(ncol = 3, nrow = times)
  MSE_matrix_2 <- matrix(ncol = 3, nrow = times)
  MSE_matrix_rl <- matrix(ncol = 3, nrow = times)
  variance_1 <- matrix(ncol = 3, nrow = N)
  variance_2 <- matrix(ncol = 3, nrow = N)
  variance_rl <- matrix(ncol = 3, nrow = N)
  matrices_prediction_1 <- lapply(1:N, function(x) matrix(0, nrow = times, ncol = 3))
  matrices_prediction_2 <- lapply(1:N, function(x) matrix(0, nrow = times, ncol = 3))
  matrices_prediction_rl <- lapply(1:N, function(x) matrix(0, nrow = times, ncol = 3))
  matrices_prediction_new_1 <- lapply(1:times, function(x) matrix(0, nrow = N, ncol = 3))
  matrices_prediction_new_2 <- lapply(1:times, function(x) matrix(0, nrow = N, ncol = 3))
  matrices_prediction_new_rl <- lapply(1:times, function(x) matrix(0, nrow = N, ncol = 3))
  for (i in 1:times){
    result <- method(num_traj,num_batch,batch,N,sig,beta,T=3)
    MSE_matrix_1[i,] <- colMeans( result[['predict_error_1']])
    MSE_matrix_2[i,] <- colMeans( result[['predict_error_2']])
    MSE_matrix_rl[i,] <- colMeans( result[['predict_error_rl']])
    for (j in 1:N){
       matrices_prediction_1[[j]][i,]<- result[['prediction_1']][j,]
       matrices_prediction_2[[j]][i,]<- result[['prediction_2']][j,]
       matrices_prediction_rl[[j]][i,]<- result[['prediction_rl']][j,]
    }
    matrices_prediction_new_1[[i]] <- result[['prediction_1']]
    matrices_prediction_new_2[[i]] <- result[['prediction_2']]
    matrices_prediction_new_rl[[i]] <- result[['prediction_rl']]
  }
  for (k in 1:N){
    variance_1[k,] <- apply(matrices_prediction_1[[k]], 2, var)
    variance_2[k,] <- apply(matrices_prediction_2[[k]], 2, var)
    variance_rl[k,] <- apply(matrices_prediction_rl[[k]], 2, var)
  }
  return(list(
    MSE_matrix_1 = MSE_matrix_1,  MSE_matrix_2 = MSE_matrix_2, MSE_matrix_rl = MSE_matrix_rl,
      variance_1 = variance_1, variance_2 = variance_2, variance_rl = variance_rl,
      matrices_prediction_new_1 = matrices_prediction_new_1, matrices_prediction_new_2 = matrices_prediction_new_2, 
      matrices_prediction_new_rl = matrices_prediction_new_rl
  ))
                                       
 }

In [5]:
N=50  ### m
times=50  ### M
num_batch <- 100 ### B
sig = 1
beta = 1

set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.3.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.3.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.3.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.3.4.txt")


set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.3.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.3.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.3.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.3.4.txt")

set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.3.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.3.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.3.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.3.4.txt")

In [6]:
N=50  ### m
times=50  ### M
num_batch <- 500 ### B
sig = 1
beta = 1

set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.2.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.2.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.2.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.2.4.txt")


set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.2.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.2.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.2.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.2.4.txt")

set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.2.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.2.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.2.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.2.4.txt")



In [7]:
N=50  ### m
times=50  ### M
num_batch <- 1000 ### B
sig = 1
beta = 1

set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.4.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.4.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.4.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.3 ,sig,times,beta)
dput(result, file = "RL_3.4.4.txt")


set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.4.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.4.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.4.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.5 ,sig,times,beta)
dput(result, file = "RL_5.4.4.txt")

set.seed(1)
num_traj=500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.4.2.txt")
set.seed(1)
num_traj=1000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.4.6.txt")
set.seed(1)
num_traj=2500 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.4.3.txt")
set.seed(1)
num_traj=5000 # n
result <- experiment_1(N,num_traj, num_batch, batch= num_traj*0.7 ,sig,times,beta)
dput(result, file = "RL_7.4.4.txt")